In [1]:
!pip install langchain transformers faiss-cpu sentence-transformers fastapi pydantic uvicorn gradio
!pip install langchain-community

import os
import numpy as np
import pandas as pd
import json
from typing import Dict, List, Any
import logging
from datetime import datetime
import torch
from tqdm.notebook import tqdm
import gradio as gr

# LangChain
from langchain.llms import HuggingFacePipeline
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate
from langchain.schema import Document

# Transformers
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline

# Configure
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 30.7/30.7 MB 12.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.3/62.3 kB 1.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.3/51.3 MB 6.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 322.2/322.2 kB 13.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.3/11.3 MB 24.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 29.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 35.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 30.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/

In [ ]:
# Sample  case data
SAMPLE_CASES = [
    {
        "title": "Privacy as a Fundamental Right",
        "citation": "Justice K.S. Puttaswamy v. Union of India (2017)",
        "court": "Supreme Court of India",
        "content": "The Supreme Court recognized the right to privacy as a fundamental right under Article 21 of the Constitution. The Court held that the right to privacy is intrinsic to life and liberty and is inherently protected under the various fundamental freedoms enshrined under Part III of the Indian Constitution."
    },
    {
        "title": "Online Defamation",
        "citation": "Swami Ramdev v. Facebook (2019)",
        "court": "Delhi High Court",
        "content": "The Delhi High Court ruled that social media platforms must remove defamatory content globally if it originates from India. The court held that defamation on social media has wide-reaching consequences and platforms cannot claim jurisdiction limitations."
    },
    {
        "title": "Environmental Protection",
        "citation": "M.C. Mehta v. Union of India (Taj Trapezium Case)",
        "court": "Supreme Court of India",
        "content": "The Supreme Court ordered industries around the Taj Mahal to either switch to natural gas or relocate outside the Taj Trapezium Zone to protect the monument from pollution damage, invoking the 'Polluter Pays Principle' and Article 21 of the Constitution."
    },
    {
        "title": "Free Speech Online",
        "citation": "Shreya Singhal v. Union of India (2015)",
        "court": "Supreme Court of India",
        "content": "The Supreme Court struck down Section 66A of the Information Technology Act, holding it violative of Article 19(1)(a) of the Constitution. The Court differentiated between advocacy and incitement, holding that only the latter can be prohibited."
    },
    {
        "title": "Digital Privacy",
        "citation": "Internet Freedom Foundation v. Union of India (2022)",
        "court": "Delhi High Court",
        "content": "The Delhi High Court ruled that digital surveillance without proper safeguards violates the right to privacy. The court mandated that any surveillance must follow due process, establish necessity, and be proportionate to the objective sought."
    }
]

# Sample statutes
SAMPLE_STATUTES = {
    "The Constitution of India, Article 21": "No person shall be deprived of his life or personal liberty except according to procedure established by law.",
    "The Information Technology Act, Section 66": "If any person, dishonestly or fraudulently, does any act referred to in section 43, he shall be punishable with imprisonment for a term which may extend to three years or with fine which may extend to five lakh rupees or with both.",
    "The Indian Penal Code, Section 499": "Whoever, by words either spoken or intended to be read, or by signs or by visible representations, makes or publishes any imputation concerning any person intending to harm, or knowing or having reason to believe that such imputation will harm, the reputation of such person, is said to defame that person.",
    "The Environment Protection Act, 1986, Section 3": "Subject to the provisions of this Act, the Central Government shall have the power to take all such measures as it deems necessary or expedient for the purpose of protecting and improving the quality of the environment and preventing, controlling and abating environmental pollution.",
    "Right to Information Act, 2005, Section 8": "Notwithstanding anything contained in this Act, there shall be no obligation to give any citizen information disclosure of which would prejudicially affect the sovereignty and integrity of India, the security, strategic, scientific or economic interests of the State, relation with foreign State or lead to incitement of an offence."
}

# demo directories
os.makedirs("case_data", exist_ok=True)
os.makedirs("model_cache", exist_ok=True)
os.makedirs("vector_db", exist_ok=True)


In [ ]:
# LLM Model

def load_language_model():
    """Load a language model suitable for legal reasoning"""
    logger.info("Loading language model...")

    # For Colab, we use a smaller model that can run efficiently
    model_name = "TheBloke/Llama-2-7B-Chat-GGML"

    try:
        # Using the HF model directly for simplicity in Colab
        tokenizer = AutoTokenizer.from_pretrained("meta-llama/Llama-2-7b-chat-hf")
        model = AutoModelForCausalLM.from_pretrained(
            "meta-llama/Llama-2-7b-chat-hf",
            device_map="auto",
            torch_dtype=torch.float16,
            load_in_8bit=True
        )

        llm_pipeline = pipeline(
            "text-generation",
            model=model,
            tokenizer=tokenizer,
            max_length=2048,
            temperature=0.3,
            top_p=0.9,
            repetition_penalty=1.1,
        )

        llm = HuggingFacePipeline(pipeline=llm_pipeline)
        logger.info("Language model loaded successfully")
        return llm

    except Exception as e:
        logger.error(f"Error loading model: {str(e)}")
        logger.info("Using a smaller model for demonstration purposes")

        def simple_llm(prompt):
            """Simple mock LLM for demonstration when hardware is limited"""
            return f"Based on the legal analysis of the case presented, I find that...\n\n" + \
                   f"The key legal issues identified are related to the primary dispute.\n\n" + \
                   f"After careful consideration of the statutes and precedents, my judgment is that...\n\n" + \
                   f"This ruling is supported by previous cases including {SAMPLE_CASES[0]['citation']}."

        return simple_llm

def setup_vector_database():
    """Set up the vector database for legal case retrieval"""
    logger.info("Setting up vector database for case law...")

    try:
        embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

        # Converting sample cases to documents
        documents = []
        for case in SAMPLE_CASES:
            documents.append(
                Document(
                    page_content=f"{case['title']}: {case['content']}",
                    metadata={
                        "citation": case["citation"],
                        "court": case["court"],
                        "title": case["title"]
                    }
                )
            )

        # Creating FAISS index
        vector_db = FAISS.from_documents(documents, embeddings)

        # Save database
        vector_db.save_local("vector_db/legal_cases")

        logger.info("Vector database created and saved successfully")
        return vector_db

    except Exception as e:
        logger.error(f"Error setting up vector database: {str(e)}")
        return None

In [ ]:
# AI Judge

class AIJudge:
    def __init__(self, llm=None, vector_db=None):
        """Initialize the AI Judge component"""
        logger.info("Initializing AI Judge system...")

        # Initialize or load models
        self.llm = llm or load_language_model()

        # Initialize or load vector database
        if vector_db is not None:
            self.vector_db = vector_db
        elif os.path.exists("vector_db/legal_cases"):
            embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
            self.vector_db = FAISS.load_local("vector_db/legal_cases", embeddings)
        else:
            self.vector_db = setup_vector_database()

        # Set up legal reasoning prompt templates
        self._setup_prompt_templates()

        logger.info("AI Judge initialized successfully")

    def _setup_prompt_templates(self):
        """Set up the prompt templates for legal reasoning"""
        # Case analysis prompt
        self.case_analysis_prompt = PromptTemplate(
            input_variables=["case_facts", "petitioner_arguments", "respondent_arguments",
                           "statutes", "precedents"],
            template="""
            You are an AI judge analyzing a legal case in the Indian legal system. Based on the
            information provided, generate a comprehensive legal analysis.

            CASE FACTS:
            {case_facts}

            PETITIONER'S ARGUMENTS:
            {petitioner_arguments}

            RESPONDENT'S ARGUMENTS:
            {respondent_arguments}

            RELEVANT STATUTES:
            {statutes}

            RELEVANT PRECEDENTS:
            {precedents}

            Provide your analysis in the following structure:
            1. IDENTIFICATION OF LEGAL ISSUES: What are the key legal questions?
            2. ANALYSIS OF FACTS: Analyze relevant facts in light of applicable law.
            3. APPLICATION OF STATUTES: Interpret and apply the relevant statutory provisions.
            4. PRECEDENT CONSIDERATION: How do the precedent cases apply?
            5. LEGAL REASONING: Provide detailed legal reasoning for your conclusion.
            6. PRELIMINARY JUDGMENT: State your preliminary judgment.
            """
        )

    def retrieve_relevant_precedents(self, case_text, k=3):
        """Retrieve relevant legal precedents for a case"""
        if self.vector_db is None:
            # Return sample data if vector DB isn't available
            return SAMPLE_CASES[:k]

        try:
            results = self.vector_db.similarity_search(case_text, k=k)
            precedents = []

            for doc in results:
                precedents.append({
                    "content": doc.page_content,
                    "citation": doc.metadata.get("citation", "Unknown"),
                    "court": doc.metadata.get("court", "Unknown"),
                    "title": doc.metadata.get("title", "Unknown")
                })

            return precedents

        except Exception as e:
            logger.error(f"Error retrieving precedents: {str(e)}")
            return SAMPLE_CASES[:k]  # Fallback to sample data

    def analyze_case(self, case_data):
        """Analyze a legal case and generate an initial judgment"""
        logger.info(f"Starting case analysis for category: {case_data.get('case_category', 'Unspecified')}")

        # Extract case data
        case_facts = case_data.get("case_facts", "")
        petitioner_arguments = case_data.get("petitioner_arguments", "")
        respondent_arguments = case_data.get("respondent_arguments", "")
        relevant_statutes = case_data.get("relevant_statutes", [])

        # Combined text for precedent search
        combined_text = f"{case_facts} {petitioner_arguments} {respondent_arguments}"

        # Retrieve relevant precedents
        precedents = self.retrieve_relevant_precedents(combined_text)
        precedent_text = "\n\n".join([
            f"PRECEDENT: {p['title']}\nCOURT: {p['court']}\nCITATION: {p['citation']}\nSUMMARY: {p['content']}"
            for p in precedents
        ])

        # Format statutes
        # For demo purposes, convert statute codes to full text if available
        statute_texts = []
        for statute in relevant_statutes:
            if statute in SAMPLE_STATUTES:
                statute_texts.append(f"{statute}: {SAMPLE_STATUTES[statute]}")
            else:
                statute_texts.append(statute)

        statutes_text = "\n".join(statute_texts)

        # Generate legal analysis
        try:
            if callable(self.llm) and not hasattr(self.llm, 'run'):
                # Simple mock LLM
                legal_analysis = self.llm(combined_text)
            else:
                # LangChain LLM
                legal_analysis_chain = LLMChain(llm=self.llm, prompt=self.case_analysis_prompt)
                legal_analysis = legal_analysis_chain.run(
                    case_facts=case_facts,
                    petitioner_arguments=petitioner_arguments,
                    respondent_arguments=respondent_arguments,
                    statutes=statutes_text,
                    precedents=precedent_text
                )

            # Process the analysis to extract structured information
            # In a real system, this would be more sophisticated
            sections = legal_analysis.split("\n\n")

            # Extract sections (simplified)
            legal_issues = "No legal issues identified."
            fact_analysis = "No fact analysis provided."
            statute_application = "No statute application provided."
            precedent_consideration = "No precedent consideration provided."
            legal_reasoning = "No detailed reasoning provided."
            preliminary_judgment = "No judgment provided."

            # Attempt to extract sections if they exist
            for section in sections:
                if "LEGAL ISSUES" in section or "IDENTIFICATION OF LEGAL ISSUES" in section:
                    legal_issues = section
                elif "ANALYSIS OF FACTS" in section:
                    fact_analysis = section
                elif "APPLICATION OF STATUTES" in section:
                    statute_application = section
                elif "PRECEDENT CONSIDERATION" in section:
                    precedent_consideration = section
                elif "LEGAL REASONING" in section:
                    legal_reasoning = section
                elif "PRELIMINARY JUDGMENT" in section:
                    preliminary_judgment = section

            # Extract a clear decision if possible
            decision = "Undetermined"
            if "in favor of petitioner" in preliminary_judgment.lower():
                decision = "In favor of Petitioner"
            elif "in favor of respondent" in preliminary_judgment.lower():
                decision = "In favor of Respondent"

            # Generate a simple confidence score
            confidence_score = 0.7  # Default mid-range confidence

            # Parse precedent citations
            precedent_citations = [
                {"citation": p["citation"], "relevance": "High" if i < 2 else "Medium"}
                for i, p in enumerate(precedents)
            ]

            return {
                "full_analysis": legal_analysis,
                "legal_issues": legal_issues,
                "fact_analysis": fact_analysis,
                "statute_application": statute_application,
                "precedent_consideration": precedent_consideration,
                "legal_reasoning": legal_reasoning,
                "preliminary_judgment": preliminary_judgment,
                "clear_decision": decision,
                "precedent_citations": precedent_citations,
                "confidence_score": confidence_score,
                "case_data": case_data
            }

        except Exception as e:
            logger.error(f"Error during case analysis: {str(e)}")
            # Return a simplified response for demo purposes
            return {
                "full_analysis": "Error during analysis",
                "legal_issues": "Error during analysis",
                "preliminary_judgment": "Unable to determine",
                "clear_decision": "Error",
                "precedent_citations": [],
                "confidence_score": 0.0,
                "error": str(e),
                "case_data": case_data
            }

In [ ]:
# AI Jury

class AIJuryMember:
    def __init__(self, perspective, llm):
        """Initialize an AI Jury member with a specific legal perspective"""
        self.perspective = perspective
        self.llm = llm
        self.perspective_descriptions = {
            "strict_interpretation": "You focus on the literal meaning of statutes and strict precedent adherence.",
            "progressive": "You consider evolving social contexts and the spirit of the law over literal interpretation.",
            "rights_based": "You prioritize fundamental rights and constitutional principles in your analysis.",
            "procedural": "You emphasize procedural correctness and due process in your analysis.",
            "economic": "You consider economic efficiency and practical implications in your legal reasoning."
        }
        self._setup_prompt_template()

    def _setup_prompt_template(self):
        """Set up the prompt template for this jury member"""
        perspective_desc = self.perspective_descriptions.get(
            self.perspective,
            "You analyze legal cases from a balanced perspective."
        )

        template = f"""
        You are an AI legal expert with a {self.perspective} perspective on the law.
        {perspective_desc}

        Review the following case and provide your analysis and opinion:

        CASE FACTS:
        {{case_facts}}

        PETITIONER'S ARGUMENTS:
        {{petitioner_arguments}}

        RESPONDENT'S ARGUMENTS:
        {{respondent_arguments}}

        INITIAL ANALYSIS BY AI JUDGE:
        {{initial_analysis}}

        Based on your {self.perspective} perspective, provide:
        1. Your agreement or disagreement with the initial analysis
        2. Your reasoning based on legal principles
        3. Your recommended verdict (for Petitioner or for Respondent)
        4. Confidence in your recommendation (Low, Medium, High)
        """

        self.prompt_template = PromptTemplate(
            input_variables=["case_facts", "petitioner_arguments",
                           "respondent_arguments", "initial_analysis"],
            template=template
        )

    def analyze(self, judge_analysis):
        """Analyze the case from this jury member's perspective"""
        try:
            # Extract relevant data
            case_data = judge_analysis["case_data"]
            case_facts = case_data.get("case_facts", "")
            petitioner_arguments = case_data.get("petitioner_arguments", "")
            respondent_arguments = case_data.get("respondent_arguments", "")

            # Get a summary of the initial analysis
            initial_analysis_summary = judge_analysis["legal_issues"] + "\n" + judge_analysis["preliminary_judgment"]

            # Generate jury member's analysis
            if callable(self.llm) and not hasattr(self.llm, 'run'):
                # Simple mock for demo
                response = f"Based on my {self.perspective} perspective, I {'agree' if np.random.random() > 0.3 else 'disagree'} with the initial analysis. " + \
                          f"I recommend a verdict for {'Petitioner' if np.random.random() > 0.5 else 'Respondent'} with {['Low', 'Medium', 'High'][np.random.randint(0, 3)]} confidence."
            else:
                chain = LLMChain(llm=self.llm, prompt=self.prompt_template)
                response = chain.run(
                    case_facts=case_facts[:500],  # Truncate for length
                    petitioner_arguments=petitioner_arguments[:300],
                    respondent_arguments=respondent_arguments[:300],
                    initial_analysis=initial_analysis_summary[:500]
                )

            # Parse the response to extract key elements (simplified)
            agreement = "Agreement" if "agree" in response.lower() else "Disagreement"
            verdict = "For Petitioner" if "petitioner" in response.lower() else "For Respondent"

            # Extract confidence
            confidence = "Medium"  # Default
            if "high confidence" in response.lower():
                confidence = "High"
            elif "low confidence" in response.lower():
                confidence = "Low"

            return {
                "perspective": self.perspective,
                "full_analysis": response,
                "agreement": agreement,
                "recommended_verdict": verdict,
                "confidence": confidence
            }

        except Exception as e:
            logger.error(f"Error in jury member analysis ({self.perspective}): {str(e)}")
            return {
                "perspective": self.perspective,
                "full_analysis": f"Error during analysis: {str(e)}",
                "agreement": "Error",
                "recommended_verdict": "Unable to determine",
                "confidence": "Low"
            }

class AIJurySystem:
    def __init__(self, llm):
        """Initialize the AI Jury system with multiple jury members"""
        self.llm = llm
        self.jury_members = {
            "strict_interpretation": AIJuryMember("strict_interpretation", llm),
            "progressive": AIJuryMember("progressive", llm),
            "rights_based": AIJuryMember("rights_based", llm),
            "procedural": AIJuryMember("procedural", llm),
            "economic": AIJuryMember("economic", llm)
        }

    def deliberate(self, judge_analysis):
        """Conduct jury deliberation on a case"""
        logger.info("Starting AI Jury deliberation process")

        # analysing the case
        jury_analyses = {}
        for name, member in self.jury_members.items():
            logger.info(f"Getting analysis from {name} jury member...")
            jury_analyses[name] = member.analyze(judge_analysis)

        # Calculate voting results
        verdict_counts = {"For Petitioner": 0, "For Respondent": 0, "Undetermined": 0}
        for analysis in jury_analyses.values():
            verdict = analysis.get("recommended_verdict", "Undetermined")
            verdict_counts[verdict] = verdict_counts.get(verdict, 0) + 1

        # Determine majority
        majority_verdict = max(verdict_counts.items(), key=lambda x: x[1])[0]
        majority_percentage = verdict_counts[majority_verdict] / len(jury_analyses) * 100

        # Generate overall confidence based on member confidence levels
        confidence_map = {"Low": 0.3, "Medium": 0.6, "High": 0.9}
        avg_confidence = sum(confidence_map.get(a.get("confidence", "Low"), 0.3) for a in jury_analyses.values()) / len(jury_analyses)

        # simplified debate
        for_petitioner = [m for m, a in jury_analyses.items() if a.get("recommended_verdict") == "For Petitioner"]
        for_respondent = [m for m, a in jury_analyses.items() if a.get("recommended_verdict") == "For Respondent"]

        debate_summary = f"AI Jury deliberation complete. {len(for_petitioner)} members ({', '.join(for_petitioner)}) " + \
                        f"voted for the Petitioner. {len(for_respondent)} members ({', '.join(for_respondent)}) " + \
                        f"voted for the Respondent. Majority verdict: {majority_verdict} ({majority_percentage:.1f}%)."

        return {
            "individual_analyses": jury_analyses,
            "verdict_counts": verdict_counts,
            "majority_verdict": majority_verdict,
            "majority_percentage": majority_percentage,
            "average_confidence": avg_confidence,
            "debate_summary": debate_summary
        }

In [ ]:
# AJDMS Main

class AJDMS:
    def __init__(self):
        """Initialize the AI Judge Decision-Making System"""
        logger.info("Initializing AJDMS...")

        self.llm = load_language_model()
        self.vector_db = setup_vector_database()

        # Initialize components
        self.ai_judge = AIJudge(self.llm, self.vector_db)
        self.ai_jury = AIJurySystem(self.llm)

        # Case storage
        self.cases = {}

        logger.info("AJDMS initialized successfully")

    def process_case(self, case_data):
        """Process a new legal case through the system"""
        logger.info("Processing new case...")

        # Generate case ID
        case_id = f"case_{datetime.now().strftime('%Y%m%d%H%M%S')}"

        # Step 1: AI Judge Analysis
        judge_analysis = self.ai_judge.analyze_case(case_data)

        # Step 2: AI Jury
        jury_deliberation = self.ai_jury.deliberate(judge_analysis)

        # Step 3: Final Verdict
        final_verdict = self._generate_final_verdict(judge_analysis, jury_deliberation)

        # Store case results
        self.cases[case_id] = {
            "case_data": case_data,
            "judge_analysis": judge_analysis,
            "jury_deliberation": jury_deliberation,
            "final_verdict": final_verdict,
            "timestamp": datetime.now().isoformat()
        }

        # Save to file
        try:
            with open(f"case_data/{case_id}.json", "w") as f:
                json.dump(self.cases[case_id], f, indent=2)
        except Exception as e:
            logger.error(f"Error saving case data: {str(e)}")

        return {
            "case_id": case_id,
            "final_verdict": final_verdict,
            "judge_analysis": judge_analysis,
            "jury_deliberation": jury_deliberation
        }

    def _generate_final_verdict(self, judge_analysis, jury_deliberation):
        """Generate the final verdict considering both Judge and Jury inputs"""
        judge_decision = judge_analysis.get("clear_decision", "Undetermined")
        jury_verdict = jury_deliberation.get("majority_verdict", "Undetermined")

        # for final decision
        if judge_decision == jury_verdict:
            final_decision = judge_decision
            decision_confidence = max(judge_analysis.get("confidence_score", 0.5),
                                    jury_deliberation.get("average_confidence", 0.5))
            reasoning = f"Both the AI Judge and AI Jury majority agree on this verdict."
        else:
            # In case of disagreement
            jury_confidence = jury_deliberation.get("average_confidence", 0.5)
            judge_confidence = judge_analysis.get("confidence_score", 0.5)

            if jury_confidence > judge_confidence:
                final_decision = jury_verdict
                decision_confidence = jury_confidence
                reasoning = f"The AI Jury's consensus carries more weight in this case."
            else:
                final_decision = judge_decision
                decision_confidence = judge_confidence
                reasoning = f"The AI Judge's analysis carries more weight in this case."

        #citations from judge's analysis
        citations = judge_analysis.get("precedent_citations", [])

        return {
            "decision": final_decision,
            "confidence": decision_confidence,
            "reasoning": reasoning,
            "citations": citations,
            "date": datetime.now().isoformat()
        }

    def get_case(self, case_id):
        """Retrieve a case by ID"""
        if case_id in self.cases:
            return self.cases[case_id]

        # Try to load from file
        try:
            with open(f"case_data/{case_id}.json", "r") as f:
                case_data = json.load(f)
                self.cases[case_id] = case_data
                return case_data
        except:
            return None


In [ ]:
# Gradio Interface

def create_gradio_interface():
    # Initialize the system
    system = AJDMS()

    # case submission function
    def submit_case(case_facts, petitioner_arguments, respondent_arguments, statutes, case_category):
        # case data
        case_data = {
            "case_facts": case_facts,
            "petitioner_arguments": petitioner_arguments,
            "respondent_arguments": respondent_arguments,
            "relevant_statutes": [s.strip() for s in statutes.split("\n") if s.strip()],
            "case_category": case_category
        }

        # Process the case
        result = system.process_case(case_data)

        # Extract results
        judge_analysis = result["judge_analysis"]
        jury_deliberation = result["jury_deliberation"]
        final_verdict = result["final_verdict"]

        judge_output = f"## AI Judge Analysis\n\n" + \
                      f"### Legal Issues\n{judge_analysis.get('legal_issues', 'Not provided')}\n\n" + \
                      f"### Preliminary Judgment\n{judge_analysis.get('preliminary_judgment', 'Not provided')}\n\n"

        jury_output = f"## AI Jury Deliberation\n\n" + \
                     f"### Debate Summary\n{jury_deliberation.get('debate_summary', 'Not provided')}\n\n" + \
                     f"### Voting Results\n" + \
                     f"For Petitioner: {jury_deliberation.get('verdict_counts', {}).get('For Petitioner', 0)}\n" + \
                     f"For Respondent: {jury_deliberation.get('verdict_counts', {}).get('For Respondent', 0)}\n\n"

        verdict_output = f"## Final Verdict\n\n" + \
                        f"### Decision\n{final_verdict.get('decision', 'Undetermined')}\n\n" + \
                        f"### Reasoning\n{final_verdict.get('reasoning', 'Not provided')}\n\n" + \
                        f"### Confidence\n{final_verdict.get('confidence', 0.0):.2f}\n\n" + \
                        f"### Citations\n" + \
                        "\n".join([f"- {c.get('citation', 'Unknown')} (Relevance: {c.get('relevance', 'Unknown')})"
                                 for c in final_verdict.get('citations', [])])

        case_id = result["case_id"]

        return judge_output, jury_output, verdict_output, case_id

    # Create the interface
    with gr.Blocks(title="AI Judge Decision-Making System") as interface:
        gr.Markdown("# AI Judge Decision-Making System (AJDMS)")
        gr.Markdown("Submit a legal case for AI analysis")

        with gr.Tab("Case Submission"):
            case_category = gr.Dropdown(
                choices=["Constitutional", "Criminal", "Civil", "Administrative", "Environmental", "Intellectual Property"],
                label="Case Category"
            )
            case_facts